# Original LSCP with constraints

Author: Huanfa Chen, Rongbo Xu

In [40]:
# %time
import numpy
import geopandas 
import pandas
import pulp
from shapely.geometry import Point
import matplotlib.pyplot as plt
# from google.colab import files
import spopt
from spopt.locate.coverage import LSCP
import time

## Import data

In [41]:
# the service distance in metres (equal to 10 miles)
service_dist = 16093.4
# the distance greater than service distance
great_dist = 20000

In [42]:
# import distance data
# the distance between existing sites and covered MSOAs
df_distance_existing_sites_covered_MSOA = pandas.read_csv('../Data/distance_df_existing_sites_MSOA.csv')

# the distance between potential sites and uncovered MSOAs
df_distance_potential_sites_all_MSOA = pandas.read_csv('../Data/distance_df_potential_sites_all_MSOA.csv')

In [43]:
df_distance_existing_sites_covered_MSOA.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122,E02002536,E122,6712.7
1,127,E02002536,E127,13881.2
2,137,E02002536,E137,13631.4
3,836,E02002536,E836,12395.3
4,838,E02002536,E838,12672.0
5,843,E02002536,E843,14449.4
6,844,E02002536,E844,14914.6
7,846,E02002536,E846,14449.4
8,849,E02002536,E849,14914.6
9,1722,E02002537,E122,7828.7


In [44]:
#总共多少个existing sites: 1600
df_distance_existing_sites_covered_MSOA.nunique()

Unnamed: 0    373286
origin_id       6408
dest_id         1600
distance      122263
dtype: int64

In [45]:
df_distance_potential_sites_all_MSOA.head(10)

,origin_id,dest_id,distance
0,E02002536,P14,7041.3
1,E02002536,P207,14950.7
2,E02002536,P354,2963.1
3,E02002537,P14,8157.3
4,E02002537,P207,16066.6
5,E02002537,P354,3153.8
6,E02002534,P123,13144.8
7,E02002534,P162,14661.7
8,E02002534,P207,12390.9
9,E02002535,P14,8752.8


In [46]:
#总共多少个potential sites: 21127
df_distance_potential_sites_all_MSOA.nunique()

origin_id      6788
dest_id       21127
distance     157528
dtype: int64

In [47]:
print(df_distance_existing_sites_covered_MSOA.columns)
print(df_distance_potential_sites_all_MSOA.columns)

Index(['Unnamed: 0', 'origin_id', 'dest_id', 'distance'], dtype='object')
Index(['origin_id', 'dest_id', 'distance'], dtype='object')


In [48]:
# combine two distance df
df_distance_existing_potential_sites_all_MSOAs = pandas.concat([df_distance_existing_sites_covered_MSOA, df_distance_potential_sites_all_MSOA], ignore_index=False)

In [49]:
df_distance_existing_potential_sites_all_MSOAs.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122.0,E02002536,E122,6712.7
1,127.0,E02002536,E127,13881.2
2,137.0,E02002536,E137,13631.4
3,836.0,E02002536,E836,12395.3
4,838.0,E02002536,E838,12672.0
5,843.0,E02002536,E843,14449.4
6,844.0,E02002536,E844,14914.6
7,846.0,E02002536,E846,14449.4
8,849.0,E02002536,E849,14914.6
9,1722.0,E02002537,E122,7828.7


## Transform data

In [50]:
# transform the distance df to matrix
ntw_dist_piv = df_distance_existing_potential_sites_all_MSOAs.pivot_table(values="distance", index="origin_id", columns="dest_id")
# transform matrix into numpy array
cost_matrix = ntw_dist_piv.to_numpy()

In [51]:
ntw_dist_piv

dest_id,E0,E1,E10,E100,E1000,E1001,E1002,E1003,E1004,E1005,...,P999,P9991,P9992,P9993,P9994,P9995,P9996,P9997,P9998,P9999
origin_id,,,,,,,,,,,,,,,,,,,,,
E02000001,NaN,NaN,NaN,NaN,NaN,5747.1,3668.0,NaN,NaN,NaN,...,10097.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E02006930,NaN,NaN,NaN,NaN,NaN,14784.0,6718.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006931,NaN,NaN,NaN,NaN,NaN,12446.5,8671.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# save the column names as a list
list_site_ID = ntw_dist_piv.columns.to_list()

In [53]:
# if an element is NA or equal to or greater than the service distance in the array, it means it is greater than the predefined service distance. Set it as great_distance
cost_matrix[cost_matrix == service_dist] = great_dist
cost_matrix[numpy.isnan(cost_matrix)] = great_dist

In [54]:
cost_matrix

array([[20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       ...,
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.]])

In [55]:
cost_matrix.shape

(6788, 22727)

## 设置Gap为0

In [ ]:
# 设置参数
num_facilities = 22727
num_demand_points = 6788

In [88]:
import gurobipy as gp
from gurobipy import GRB

m_gap = gp.Model('facility_location_gap')
# 添加决策变量
select_gap = m_gap.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
# 设置限制条件
    # 每个i在距离x(16093.4)内至少被1个j覆盖
m_gap.addConstrs((gp.quicksum(select_gap[j] for j in range(num_facilities) if cost_matrix[i,j] < 16093.4) >= 1  for i in range(num_demand_points)), name='Demand_a_gap')
    # existing facility数量等于107
m_gap.addConstr(gp.quicksum(select_gap[j] for j in range(1600)) == 107, name = 'Demand_b_gap')

<gurobi.Constr *Awaiting Model Update*>

In [89]:
m_gap.setObjective(gp.quicksum(select_gap[j] for j in range(num_facilities)), GRB.MINIMIZE)
m_gap.setParam(GRB.Param.PoolSolutions, 100)
m_gap.setParam(GRB.Param.PoolGap, 0)
m_gap.setParam(GRB.Param.PoolSearchMode, 2)
m_gap.optimize()

Set parameter PoolSolutions to value 100
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6789 rows, 22727 columns and 2612668 nonzeros
Model fingerprint: 0x404cbce7
Variable types: 0 continuous, 22727 integer (22727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 490.0000000
Presolve removed 2191 rows and 1 columns
Presolve time: 4.33s
Presolved: 4598 rows, 22726 columns, 2336729 nonzeros
Variable types: 0 continuous, 22726 integer (22724 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1000000e+02   4.696750e+03   0.000000e+00      5s
    8119    3.0946566e+02   0.000000e+00   0.000000e+00     

In [90]:
nSolutions_gap = m_gap.SolCount
print('Number of solutions found: ' + str(nSolutions_gap))

Number of solutions found: 52


In [92]:
# Print objective values of solutions
for e in range(nSolutions_gap):
    m_gap.setParam(GRB.Param.SolutionNumber, e)
    print('%g ' % m_gap.PoolObjVal, end='')
    if e % 15 == 14:
        print('')
print('')

313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 313 313 313 313 313 313 313 313 
313 313 313 313 313 313 313 


In [93]:
summary_gap = pandas.DataFrame(columns=['existing_count', 'potential_count'], index=range(nSolutions_gap))

In [94]:
for n in range(nSolutions_gap):
    m_gap.setParam(GRB.Param.SolutionNumber, n)
    
    #一共1600个existing facilities；计算序号小于等于1599的有多少个
    summary_gap.iloc[n,0] = sum(select_gap[j].Xn for j in range(1600)) 

    #计算potential sites数量
    summary_gap.iloc[n,1] = sum(select_gap[j].Xn for j in range(1600, num_facilities)) 

In [95]:
summary_gap

,existing_count,potential_count
0,107.0,206.0
1,107.0,206.0
2,107.0,206.0
3,107.0,206.0
4,107.0,206.0
5,107.0,206.0
6,107.0,206.0
7,107.0,206.0
8,107.0,206.0
9,107.0,206.0


In [96]:
value_gap = []
for e in range(nSolutions_gap):
    m_gap.setParam(GRB.Param.SolutionNumber, e)
    value_gap.append([select_gap[j].Xn for j in range(num_facilities)])

In [97]:
value_gap[50]

[1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.

## 设置现存为108

In [105]:
m_b = gp.Model('facility_location_b')
# 添加决策变量
select_b = m_b.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
# 设置限制条件
    # 每个i在距离x(16093.4)内至少被1个j覆盖
m_b.addConstrs((gp.quicksum(select_b[j] for j in range(num_facilities) if cost_matrix[i,j] < 16093.4) >= 1  for i in range(num_demand_points)), name='Demand_a')
    # existing facility数量等于108
m_b.addConstr(gp.quicksum(select_b[j] for j in range(1600)) == 108, name = 'Demand_b')
m_b.addConstr(gp.quicksum(select_b[j] for j in range(num_facilities)) == 313, name = 'Demand_c')

<gurobi.Constr *Awaiting Model Update*>

In [106]:
m_b.setObjective(313)
m_b.setParam(GRB.Param.PoolSolutions, 100)
m_b.setParam(GRB.Param.PoolGap, 0)
m_b.setParam(GRB.Param.PoolSearchMode, 2)
m_b.optimize()

Set parameter PoolSolutions to value 100
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6790 rows, 22727 columns and 2635395 nonzeros
Model fingerprint: 0x0e2b3275
Variable types: 0 continuous, 22727 integer (22727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2194 rows and 1 columns
Presolve time: 4.58s
Presolved: 4596 rows, 22726 columns, 2357720 nonzeros
Variable types: 0 continuous, 22726 integer (22724 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1300000e+02   4.897500e+03   0.000000e+00      7s
    5867    3.1300025e+02   3.500974e+02   0.000000e+00     10s
    6463    3.1300000e+02   0.000000e+00   0